In [6]:
import torch
import torch.utils.data as data
import pandas as pd
from src.error_measures import collect_stats, print_average_stats
from src.models import MultiLayerANN, SynapticSNN, LeakySNN, DoubleLeakySNN
from src.train_model import training_loop

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [8]:
def multiple_trainings(dataset, trials=3, hidden=3000):
    err_model1 = [0., 0., 0., 0., 0., 0.]
    err_model2 = [0., 0., 0., 0., 0., 0.]
    err_model3 = [0., 0., 0., 0., 0., 0.]
    err_model4 = [0., 0., 0., 0., 0., 0.]

    for i in range(trials):
        # Prepare data loaders.
        train_dataset, test_dataset = torch.utils.data.random_split(dataset, (round(0.8 * len(dataset)), round(0.2 * len(dataset))))
        train_loader = data.DataLoader(train_dataset, batch_size=32, shuffle=True, drop_last=True)
        test_loader = data.DataLoader(test_dataset, batch_size=256, shuffle=False, drop_last=False)

        # Prepare models.
        model1 = MultiLayerANN(num_inputs=8, num_hidden=hidden, num_outputs=1)
        model2 = LeakySNN(num_inputs=8, num_hidden=hidden, num_outputs=1, num_steps=25)
        model3 = SynapticSNN(num_inputs=8, num_hidden=hidden, num_outputs=1, num_steps=50)
        model4 = DoubleLeakySNN(num_inputs=8, num_hidden=hidden, num_outputs=1, num_steps=20)

        # Train models.
        print(f'------------------------------------\nTraining {i+1}')
        training_loop(model1, train_loader, device, num_epochs=120) #90
        training_loop(model2, train_loader, device, num_epochs=62) #50
        training_loop(model3, train_loader, device, num_epochs=30) #25
        training_loop(model4, train_loader, device, num_epochs=37) #35

        # Gather results.
        collect_stats(model1, err_model1, test_loader, device)
        collect_stats(model2, err_model2, test_loader, device)
        collect_stats(model3, err_model3, test_loader, device)
        collect_stats(model4, err_model4, test_loader, device)

        # Print average results.
    print('------Simple ANN ------')
    print_average_stats(err_model1, trials)
    print('------LIF SNN ------')
    print_average_stats(err_model2, trials)
    print('------Synaptic SNN ------')
    print_average_stats(err_model3, trials)
    print('------Double LIF SNN ------')
    print_average_stats(err_model4, trials)

## Data 6h

In [9]:
data06 = pd.read_csv("../data/processed/target06.csv")
data06.drop(columns=["weekday"], inplace=True)
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data06.values[:,-1]).float())  # with normalization

In [10]:
multiple_trainings(all_data, trials=1, hidden=3000)

------------------------------------
Training 1


  6%|▋         | 4/62 [00:49<12:02, 12.46s/it]


KeyboardInterrupt: 

## Data 12h

In [ ]:
data12 = pd.read_csv("../data/processed/target12.csv")
data12.drop(columns=["weekday"], inplace=True)
all_data = data.TensorDataset(torch.from_numpy((data12.values[:,:-1] - data12.values[:,:-1].min(0)) / data12.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data12.values[:,-1]).float())  # with normalization

In [ ]:
multiple_trainings(all_data, trials=2, hidden=3000)

## Data 24h

In [ ]:
data24 = pd.read_csv("../data/processed/target24.csv")
data24.drop(columns=["weekday"], inplace=True)
all_data = data.TensorDataset(torch.from_numpy((data24.values[:,:-1] - data24.values[:,:-1].min(0)) / data24.values[:,:-1].ptp(0)).float(),
                              torch.from_numpy(data24.values[:,-1]).float())  # with normalization

In [ ]:
multiple_trainings(all_data, trials=2, hidden=3000)